In [1]:
#saved file with    _ is from PSG
#saved file without _ is from PSG2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random
from sklearn import datasets
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from deslib.des import METADES
from deslib.des import KNORAE

import warnings
warnings.filterwarnings("ignore")

from REC import *
random.seed(2021)
rng = np.random.RandomState(42)

In [2]:
#simulation
# a=np.random.rand(500,2)
# b=np.random.rand(500,2)
# w1=np.array([[-0.55,0],[0,0.23]])
# w2=np.array([[1.55,0],[0,0.77]])
# y=np.dot(a,w1)+np.dot(b,w2)
# #y=NCImax(y)

# data=np.concatenate((a,b)).reshape(2, *np.array(a).shape)
# nrep=30
# lmda=0.001
# alpha=0.00001
# M=2
# K=2
# finalW=full(nrep,M,K,data,y)
# finalW=PSG(finalW,M,K,data,y,alpha,lmda)
# print(finalW)

# data=np.hstack([a,b])
# reg=LinearRegression(fit_intercept=False).fit(data, y)
# reg.coef_.T

In [3]:
# from matplotlib import pyplot
# from pandas import DataFrame

# X, y = make_moons(n_samples=1000,noise=0.2)
# df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
# colors = {0:'red', 1:'blue'}
# fig, ax = pyplot.subplots()
# grouped = df.groupby('label')
# for key, group in grouped:
#     group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
# pyplot.show()

In [4]:
rep=30
ans=np.zeros((rep,8))

for i in range(rep):
    K=2 #number of classes
    x, y = make_moons(n_samples=1000,noise=0.4)
    dim=x.shape[1]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
    y_test=y_test+1
    y_train_keep=y_train
    encoder = LabelEncoder()
    encoder.fit(y_train)
    encoded_Y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_Y)
    
    model = XGBClassifier()
    model.fit(x_train, y_train_keep) 
    P=model.predict_proba(x_train)
    P=NCI2(P)
    
    model1 = GaussianNB()
    model1.fit(x_train, y_train_keep)
    P1=model1.predict_proba(x_train)
    P1=NCI2(P1)
    
    model2=LogisticRegression(random_state=0).fit(x_train, y_train_keep)
    P2=model2.predict_proba(x_train)
    P2=NCI2(P2)
    
#     model3 = Sequential()
#     model3.add(Dense(40, input_dim=dim, activation='relu'))
#     model3.add(Dense(20, activation='relu'))
#     model3.add(Dense(K, activation='softmax'))
#     model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model3.fit(x_train, y_train, epochs=150, batch_size=10,verbose=0)
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model3 = Sequential()
    model3.add(Dense(40, input_dim=dim, activation='relu'))
    model3.add(Dense(20, activation='relu'))
    model3.add(Dense(K, activation='softmax'))
    model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model3.fit(x_train, y_train, epochs=15, validation_split=0.3, callbacks=[callback], verbose=False)

    P3=model3.predict(x_train)
    P3=NCI2(P3)
    
    M=4 #number of candidates
    #P0=NCIy(y_train,x_train,y_train_keep)
    #P0=NCI2(P0)
    P0=NCI2(y_train)
    
    T=model.predict_proba(x_test)
    T1=model1.predict_proba(x_test)
    T2=model2.predict_proba(x_test)
    T3=model3.predict(x_test)
    T=NCI2(T)
    T1=NCI2(T1)
    T2=NCI2(T2)
    T3=NCI2(T3)
    
    data=np.concatenate((P,P1,P2,P3)).reshape(M, *np.array(P1).shape)
    nrep=100
    lmda=0.001
    alpha=0.001 #0.01 to jump larger step
    initW=full(nrep,M,K,data,P0)
    finalW1=PSG(initW,M,K,data,P0,alpha,lmda)
    finalW2=PSG2(initW,M,K,data,P0,alpha)
    #lp1=pieceM(data,finalW1)
    #lp2=pieceM(data,finalW2)
    #use different nk since we don't know truth for test data
#     T0=np.array(y_test)
#     nk0=find_nK2(T0,K)
#     T0=NCI(K,T0,nk0)
    dt=np.concatenate((T,T1,T2,T3)).reshape(M, *np.array(T1).shape)
    lp3=pieceM(dt,finalW1)
    lp4=pieceM(dt,finalW2)
    
    ###################
    X_train, X_dsel, y_train, y_dsel = train_test_split(x_train, y_train_keep,
                                                    test_size=0.5,
                                                    random_state=rng)
    kne = KNORAE(random_state=rng)
    meta = METADES(random_state=rng)
    # Fitting the des techniques
    kne.fit(X_dsel, y_dsel)
    meta.fit(X_dsel, y_dsel)
    
    ans[i,:]=[accuracy_score(y_test, labelP1(T)),accuracy_score(y_test, labelP1(T1)),
              accuracy_score(y_test, labelP1(T2)),accuracy_score(y_test, labelP1(T3)),
              kne.score(x_test, y_test-1), meta.score(x_test, y_test-1),
              accuracy_score(y_test, labelP1(lp3)),accuracy_score(y_test, labelP1(lp4))]

[20:41:48] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:41:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:41:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:42:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [5]:
print(np.mean(ans,axis=0))

[0.82966667 0.82733333 0.82866667 0.83522222 0.81788889 0.81888889
 0.84844444 0.84833333]


In [6]:
np.std(ans, axis=0)

array([0.0237479 , 0.02308439, 0.02323152, 0.02276585, 0.02173039,
       0.01923217, 0.02156357, 0.02159818])

In [8]:
fileName="synthetic"+".csv"
np.savetxt(fileName, ans, delimiter=",")